# Ghosting Cell Data

In [1]:
# Clone the repository
!git clone https://github.com/FERMat-ML/OMatG.git
%cd OMatG

# Install the package and dependencies
!pip install -e .

fatal: destination path 'OMatG' already exists and is not an empty directory.
/Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/data-gen/OMatG
Obtaining file:///Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/data-gen/OMatG
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for omg (pyproject.toml) ... done
  Created wheel for omg: filename=omg-1.0.1-0.editable-py3-none-any.whl size=13624 sha256=6d4b8a96580c422f793ff39a2aa2211b274e1f58b3b046fffb23c60c473b4202
  Stored in directory: /private/var/folders/xj/8d93_b314hd1dzzj26rqjrqr0000gp/T/pip-ephem-wheel-cache-mlw3z7nw/wheels/9a/ad/9a/849a4145168353189de2dde7f0e42e8cea5ef2f7b82c57c1d3
Successfully built omg
  Attempting uninstall: omg
    Found existing installation: omg 1.0.1
    Uninstalling omg-1.0.1:
      Successfully uninstalled omg-1

In [2]:
!pip install numpy freud-analysis


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# import os
# import torch

# # Need to edit the file_path to where Voronoi.py is saved
# # file_path_in_drive = 'My Drive/Colab Notebooks' # ethan's
# file_path_in_drive = 'My Drive/martiniani_lab_research' # kosta's
# full_path = os.path.join('/content/drive', file_path_in_drive)
# sys.path.append(full_path)
import sys
import os
%cd /Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/data-gen
notebook_dir = os.getcwd()
print(notebook_dir)
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)
print(sys.path)

import numpy as np
from voronoi_weighted import VoronoiPhantomCellGenerator
import freud
print("Successfully imported Voronoi!")

/Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/data-gen
/Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/data-gen
['/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/ghosting-atoms/omg-venv/lib/python3.11/site-packages', '__editable__.omg-1.0.1.finder.__path_hook__', '/Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/data-gen']
Successfully imported Voronoi!


In [4]:
# create Voronoi Generator
VoronoiGenerator = VoronoiPhantomCellGenerator(
  desired_atom_count=20,
  dist_eval="min"
)

In [5]:
!pip show omg
print(f"{'-'*100}")
!pip show torch-geometric

Name: omg
Version: 1.0.1
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: /Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/ghosting-atoms/omg-venv/lib/python3.11/site-packages
Editable project location: /Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/data-gen/OMatG
Requires: ase, lightning, lmdb, loguru, matminer, matplotlib, numpy, pymatgen, pytest, pyyaml, scikit-learn, scipy, smact, spglib, torch, torch_geometric, torch_scatter, torchdiffeq, torchsde, tqdm, wandb
Required-by: 
----------------------------------------------------------------------------------------------------
Name: torch-geometric
Version: 2.7.0
Summary: Graph Neural Network Library for PyTorch
Home-page: https://pyg.org
Author: 
Author-email: Matthias Fey <matthias@pyg.org>
License-Expression: MIT
Location: /Users/kosta/Documents/Research/martiniani/omatg_TomEgg/code/ghosting-atoms/omg-venv/lib/python3.11/site-packages
Requires: aiohttp, fsspec, jinja2, numpy, psutil, pypar

In [6]:
import torch
import lmdb
import ase
import torch_geometric # problematic import, when debugging this, reinstall torch, etc.
# pip uninstall torch torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
# pip install torch==2.8.0
# pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.8.0+cpu.html
# pip install torch-geometric
import lightning
import loguru
import monty

In [7]:
from omg.datamodule import DataModule

# Create DataModule instances and load from LMDB
train_data = DataModule()
train_data.add_from_lmdb("./OMatG/omg/data/mp_20/train.lmdb")

val_data = DataModule()
val_data.add_from_lmdb("./OMatG/omg/data/mp_20/val.lmdb")

print(f"\nTraining set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")


2025-11-05 19:22:43.871 | INFO     | omg.datamodule.datamodule:add_from_lmdb:864 - Reusing existing LMDB file: dataset_d4eb2861e5c6ef0ecbbe1a623fffd444.lmdb
2025-11-05 19:22:44.039 | INFO     | omg.datamodule.datamodule:add_from_lmdb:864 - Reusing existing LMDB file: dataset_320b45805b142f182e8c2df221a22de4.lmdb



Training set size: 27136
Validation set size: 9047


In [8]:
from omg.datamodule.dataloader import OMGTorchDataset, OMGDataModule
from torch_geometric.loader import DataLoader

# Assuming you have train_data and val_data as DataModule instances
# Convert to PyTorch Geometric datasets
train_dataset = OMGTorchDataset(
    dataset=train_data,           # Your DataModule instance
    convert_to_fractional=False,
    niggli=False                  # Apply Niggli cell reduction (optional)
)

val_dataset = OMGTorchDataset(
    dataset=val_data,
    convert_to_fractional=False,
    niggli=False
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Train dataset type: {type(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")


Train dataset size: 27136
Train dataset type: <class 'omg.datamodule.dataloader.OMGTorchDataset'>
Val dataset size: 9047


# Data Vizualizer

In [9]:
import pandas as pd
import plotly.graph_objects as go

def plot_crystal_with_points(single_data, idx):
  """
  Plots a crystal structure with original and ghost points.

  Args:
      single_data: A data object containing cell, positions, and species.
      idx: The index of the sample being plotted.
  """
  cell = single_data.cell[0].numpy()
  x_vec, y_vec, z_vec = cell[0], cell[1], cell[2]

  # Assuming that positions are in cartesian coords not fractional coords
  positions = single_data.pos.numpy()
  species = single_data.species.numpy()

  # Create DataFrames for both original and ghost points
  df_points = pd.DataFrame(positions, columns=['x', 'y', 'z'])
  df_points['species'] = species
  df_points['type'] = df_points['species'].apply(lambda s: 'Original Points' if s != -1 else 'Ghost Points')

  # --- START of programmatic color and size mapping ---
  import plotly.express as px
  import matplotlib.pyplot as plt

  # Normalize atomic numbers to the range [0, 1] for the color scale
  # We add 1 to handle the ghost atom at -1 correctly
  norm = plt.Normalize(vmin=-1, vmax=df_points['species'].max())
  
  # Get a color from the 'Viridis' color scale
  colorscale = px.colors.sequential.Viridis
  df_points['color'] = [px.colors.sample_colorscale(colorscale, val)[0] for val in norm(df_points['species'])]
  
  # Make ghost atoms red
  df_points.loc[df_points['species'] == -1, 'color'] = 'red'

  # Programmatically determine size (e.g., linear scaling)
  # Base size of 6, increasing with atomic number
  df_points['size'] = 6 + (df_points['species'] + 1) * 0.75
  # --- END of programmatic color and size mapping ---

  # Create the custom hover text
  df_points['hover_text'] = df_points.apply(
      lambda row: f"A_n: {row['species']}<br>Pos: ({row['x']:.2f}, {row['y']:.2f}, {row['z']:.2f})",
      axis=1
  )


  # Create figure
  fig = go.Figure()

  # Add scatter points for all points, using mapped color and size
  fig.add_trace(go.Scatter3d(
      x=df_points['x'],
      y=df_points['y'],
      z=df_points['z'],
      mode='markers',
      marker=dict(
          size=df_points['size'],
          color=df_points['color'],
          opacity=0.6
      ),
      name='Atoms', # Combined legend entry
      hovertext=df_points['hover_text'],
      hoverinfo='text'
  ))


  # Define the 8 vertices of the parallelepiped
  origin = np.array([0, 0, 0])
  vertices = [
      origin,                          # 0: (0,0,0)
      x_vec,                          # 1: x_vec
      y_vec,                          # 2: y_vec
      z_vec,                          # 3: z_vec
      x_vec + y_vec,                  # 4: x_vec + y_vec
      x_vec + z_vec,                  # 5: x_vec + z_vec
      y_vec + z_vec,                  # 6: y_vec + z_vec
      x_vec + y_vec + z_vec          # 7: x_vec + y_vec + z_vec
  ]

  # Define the 12 edges (pairs of vertex indices to connect)
  edges = [
      (0, 1), (0, 2), (0, 3),  # edges from origin
      (1, 4), (1, 5),          # edges from x_vec
      (2, 4), (2, 6),          # edges from y_vec
      (3, 5), (3, 6),          # edges from z_vec
      (4, 7), (5, 7), (6, 7)   # edges to opposite corner
  ]

  # Draw each edge
  for i, j in edges:
      v1, v2 = vertices[i], vertices[j]
      fig.add_trace(go.Scatter3d(
          x=[v1[0], v2[0]],
          y=[v1[1], v2[1]],
          z=[v1[2], v2[2]],
          mode='lines',
          line=dict(color='black', width=3),
          showlegend=False,
          hoverinfo='none'
      ))

  fig.update_layout(
      title=f'Sample {idx}: Newly Loaded Data',
      width=500,  # Width in pixels
      height=500,  # Height in pixels
      autosize=False,  # Disable autosize to use fixed dimensions
      scene=dict(
          xaxis_title='X',
          yaxis_title='Y',
          zaxis_title='Z'
      )
  )

  fig.show()


#### Data Example

In [11]:
i=7
single_data = train_dataset[i]
cell = single_data.cell[0].numpy()
x_vec, y_vec, z_vec = cell[0], cell[1], cell[2]

# Assuming that positions are in cartesian coords not fractional coords
positions = single_data.pos.numpy()
atomic_numbers = single_data.species.numpy()
new_points = positions.copy()
new_atomic_numbers = atomic_numbers.copy()

# Calculate iterations once
iterations = 20 - single_data.n_atoms.item()

# Generate ghost atoms
for _ in range(iterations):
    next_point = VoronoiGenerator._get_next_point(
        points=new_points,
        atomic_numbers=new_atomic_numbers, # Pass the updated atomic_numbers
        x_vec=x_vec,
        y_vec=y_vec,
        z_vec=z_vec
      )
    new_atomic_numbers = np.append(new_atomic_numbers, 0)
    new_points = np.vstack([new_points, next_point])

# --- Create data object for plotting ---
from omg.datamodule.dataloader import OMGData
import torch

temp_data_for_plotting = OMGData()
temp_data_for_plotting.cell = torch.from_numpy(cell).unsqueeze(0)
temp_data_for_plotting.pos = torch.from_numpy(new_points)
temp_data_for_plotting.species = torch.from_numpy(new_atomic_numbers)

print(f"Data for plotting sample {i} has been generated.")

plot_crystal_with_points(temp_data_for_plotting, i)

Data for plotting sample 7 has been generated.


## New Dataset Generation

In [12]:
new_data_list = []
import pickle
import numpy as np

for idx in range(10):

  single_data = train_dataset[idx]
  cell = single_data.cell[0].numpy()
  x_vec, y_vec, z_vec = cell[0], cell[1], cell[2]

  # Assuming that positions are in cartesian coords not fractional coords
  positions = single_data.pos.numpy()
  atomic_numbers = single_data.species.numpy()
  new_points = positions.copy()
  new_atomic_numbers = atomic_numbers.copy()


  # Calculate iterations once
  iterations = 20 - single_data.n_atoms.item()

  # Generate ghost atoms
  for i in range(iterations):
      # Define the dictionary of inputs for the function
      inputs_for_crash_analysis = {
          'points': new_points,
          'atomic_numbers': new_atomic_numbers,
          'x_vec': x_vec,
          'y_vec': y_vec,
          'z_vec': z_vec
      }
      
      next_point = VoronoiGenerator._get_next_point(points=new_points, atomic_numbers=new_atomic_numbers, x_vec=x_vec, y_vec=y_vec, z_vec=z_vec)
      
      if np.any(np.isnan(next_point)):
          print("NaN value detected, breaking loop.")
          break

      new_atomic_numbers = np.append(new_atomic_numbers, -1)
      new_points = np.vstack([new_points, next_point])


  # Update pos, n_atoms, batch
  single_data.pos = torch.from_numpy(new_points).to(dtype=torch.float64)
  single_data.n_atoms = torch.tensor(20, dtype=torch.long)
  single_data.batch = torch.zeros(20, dtype=torch.long)  # All atoms belong to config 0

  # Update species efficiently
  num_ghosts = iterations
  ghost_species = torch.full((num_ghosts,), -1, dtype=torch.long)
  # Ensure single_data.species is a tensor before concatenating
  if not isinstance(single_data.species, torch.Tensor):
      single_data.species = torch.tensor(single_data.species, dtype=torch.long)
  single_data.species = torch.cat([single_data.species.long(), ghost_species])

  new_data_list.append(single_data)

  if idx == 1:
    plot_crystal_with_points(single_data, idx)

In [13]:
from torch_geometric.data import InMemoryDataset
import torch
import os

# Create the directory if it doesn't exist
os.makedirs('processed', exist_ok=True)

# Save as processed data
torch.save(new_data_list, 'processed/modified_data.pt')

In [14]:
import torch

# Load the dataset
new_data_list = torch.load('processed/modified_data.pt', weights_only=False)

# Check what you loaded
print(f"Loaded {len(new_data_list)} samples")
print(f"First sample: {new_data_list[0]}")

Loaded 10 samples
First sample: OMGData(
  n_atoms=20,
  batch=[20],
  species=[20],
  cell=[1, 3, 3],
  pos=[20, 3],
  property={}
)


In [15]:
# Iterate through all samples in the loaded data and plot
for idx, single_data in enumerate(new_data_list):
  plot_crystal_with_points(single_data, idx)

# Atoms -> XYZ -> LMDB

In [19]:
from ase import Atoms
from ase.io import write
from typing import Union
from pathlib import Path
from omg.datamodule.dataloader import OMGData
import torch
import os

def xyz_saver(data: Union[OMGData, list[OMGData]], filename: Path) -> None:
    """
    Save structures from OMGData instances to an xyz file.
    Correctly handles a list of single data objects and converts ghost
    atom species from -1 to 0 for compatibility with the XYZ format.
    """
    if not filename.suffix == ".xyz":
        raise ValueError("The filename must have the suffix '.xyz'.")
    if not isinstance(data, list):
        data = [data]
        
    atoms_list = []
    for d in data:
        # Get species and convert to numpy array
        species_np = d.species.cpu().numpy()
        
        # Replace -1 with 0 for ghost atoms (XYZ format compatibility)
        # 0 corresponds to the 'X' dummy atom in ASE.
        species_np[species_np == -1] = 0
        
        atoms_list.append(Atoms(
            numbers=species_np, 
            positions=d.pos.cpu().numpy(),
            cell=d.cell[0].cpu().numpy(), 
            pbc=True
        ))
        
    # Overwrite the file with the new list of atoms.
    os.makedirs(filename.parent, exist_ok=True)
    write(filename, atoms_list, append=False)

In [20]:
from pathlib import Path
import lmdb
import pickle
from ase.io import read
import torch
from typing import Union
from omg.datamodule.dataloader import OMGData

def convert_xyz_to_lmdb(input_xyz_file: Path, output_lmdb_file: Path) -> None:
    """Converts an ASE-readable xyz file to a single-file LMDB database."""
    if not input_xyz_file.exists():
        print(f"Error: Input file not found at {input_xyz_file}")
        return

    try:
        atoms_list = read(input_xyz_file, index=':')
        print(f"Read {len(atoms_list)} structures from '{input_xyz_file}'")
    except Exception as e:
        print(f"Error reading XYZ file: {e}")
        return

    env = lmdb.open(str(output_lmdb_file), subdir=False, map_size=int(1e12))
    with env.begin(write=True) as txn:
        for i, atoms in enumerate(atoms_list):
            key = str(i).encode('utf-8')
            value = {
                'atomic_numbers': torch.from_numpy(atoms.get_atomic_numbers()),
                'pos': torch.from_numpy(atoms.get_positions()).to(dtype=torch.float64),
                'cell': torch.from_numpy(atoms.get_cell()[:]).to(dtype=torch.float64),
                'pbc': torch.tensor(atoms.get_pbc(), dtype=torch.bool)
            }
            serialized_value = pickle.dumps(value)
            txn.put(key, serialized_value)
    env.close()
    print(f"Successfully created single-file LMDB: '{output_lmdb_file}'")

def save_atoms_to_lmdb(data: Union[OMGData, list[OMGData]], xyz_filename: Path, lmdb_filename: Path) -> None:
    """
    Saves atomic data to a single-file LMDB by first saving to an
    intermediate XYZ file and then converting.
    """
    print(f"--- Starting save process: Atoms -> XYZ -> LMDB ---")
    # Step 1: Save the data to an XYZ file
    xyz_saver(data, xyz_filename)
    print(f"Successfully saved intermediate file: '{xyz_filename}'")
    
    # Step 2: Convert the XYZ file to an LMDB file
    convert_xyz_to_lmdb(xyz_filename, lmdb_filename)
    print(f"--- Save process complete ---")


In [12]:
# !pip install atom3d

In [13]:
# Save the atoms to xyz file
assert new_data_list is not None
assert False not in [isinstance(data, OMGData) for data in new_data_list]
assert len(new_data_list) == 10
xyz_saver(new_data_list, Path('./test-save/test.xyz'))
print(f"Successfully saved the atoms to xyz file")

NameError: name 'new_data_list' is not defined

In [20]:
# --- Define File Paths ---
xyz_filepath = Path('./test-save/test.xyz')
lmdb_filepath = Path('./test-save/test.lmdb')

# --- Execute the full saving pipeline ---
save_atoms_to_lmdb(new_data_list, xyz_filepath, lmdb_filepath)


--- Starting save process: Atoms -> XYZ -> LMDB ---
Successfully saved intermediate file: 'test-save/test.xyz'
Read 10 structures from 'test-save/test.xyz'
Successfully created single-file LMDB: 'test-save/test.lmdb'
--- Save process complete ---


# Test Loading Newly Saved LMDB and Vis Them

In [21]:
# Load the newly created LMDB file
test_data = DataModule()
test_data.add_from_lmdb("./test-save/test.lmdb")

print(f"\nLoaded test set size: {len(test_data)}")

# Convert to a PyTorch Geometric dataset to make it iterable for plotting
test_dataset = OMGTorchDataset(
    dataset=test_data,
    convert_to_fractional=False,
    niggli=False
)
print(f"Iterable test dataset size: {len(test_dataset)}")

# Iterate through all samples in the newly loaded test data and plot
idx = 7
print(f"--- Plotting Sample {idx} from test.lmdb ---")
plot_crystal_with_points(test_dataset[idx], idx)


2025-11-03 17:13:50.859 | INFO     | omg.datamodule.datamodule:add_from_lmdb:864 - Reusing existing LMDB file: dataset_e8d30516240d30a09ee67873245d1cc8.lmdb



Loaded test set size: 10
Iterable test dataset size: 10
--- Plotting Sample 7 from test.lmdb ---


In [22]:
import torch

print("--- Running validation tests: Comparing original data with loaded data ---")

# The test_dataset comes from the previously loaded LMDB.
# The new_data_list is the original data in memory.
assert len(new_data_list) == len(test_dataset), f"Mismatch in number of samples! Original: {len(new_data_list)}, Loaded: {len(test_dataset)}"

for i in range(len(new_data_list)):
    original_data = new_data_list[i]
    loaded_data = test_dataset[i]

    # 1. Compare atomic numbers (species)
    # We must account for the -1 -> 0 conversion for ghost atoms in the xyz_saver.
    original_species = original_data.species.clone()  # Use clone to avoid modifying the original list in memory
    original_species[original_species == -1] = 0
    assert torch.equal(original_species.long(), loaded_data.species.long()), f"Data mismatch in Sample {i}: Species are different."

    # 2. Compare positions (floating point data)
    assert torch.allclose(original_data.pos, loaded_data.pos, atol=1e-6), f"Data mismatch in Sample {i}: Positions are different."

    # 3. Compare cell vectors (floating point data)
    assert torch.allclose(original_data.cell, loaded_data.cell, atol=1e-6), f"Data mismatch in Sample {i}: Cell vectors are different."

    print(f"Sample {i}: OK -- distance: {torch.norm(original_data.pos - loaded_data.pos)}")

print("\n✅ All 10 samples successfully verified! The data loaded from LMDB is identical to the original data.")



--- Running validation tests: Comparing original data with loaded data ---
Sample 0: OK -- distance: 2.1994283750920674e-08
Sample 1: OK -- distance: 1.829887978402069e-08
Sample 2: OK -- distance: 2.2056002165883905e-08
Sample 3: OK -- distance: 2.1674435347108823e-08
Sample 4: OK -- distance: 1.9086601199885465e-08
Sample 5: OK -- distance: 2.059461944130234e-08
Sample 6: OK -- distance: 2.283173237792461e-08
Sample 7: OK -- distance: 2.1951267674475473e-08
Sample 8: OK -- distance: 2.1293322039394226e-08
Sample 9: OK -- distance: 1.9548439724702773e-08

✅ All 10 samples successfully verified! The data loaded from LMDB is identical to the original data.


# Now, We'll do this On the Whole Dataset

In [16]:
from tqdm import tqdm
import numpy as np
import torch
import pickle

# This list will store all the processed data objects
ghosted_val_data_list = []

# Use tqdm to create a progress bar for the loop
print(f"Starting ghost atom generation for {len(val_dataset)} samples...")

for single_data in tqdm(val_dataset):
    # --- 1. Extract initial data ---
    cell = single_data.cell[0].numpy()
    x_vec, y_vec, z_vec = cell[0], cell[1], cell[2]
    
    positions = single_data.pos.numpy()
    atomic_numbers = single_data.species.numpy()
    
    new_points = positions.copy()
    new_atomic_numbers = atomic_numbers.copy()

    # --- 2. Generate ghost atoms ---
    iterations = 20 - single_data.n_atoms.item()
    if iterations > 0:
        for _ in range(iterations):
            # --- Save the inputs for debugging ---
            debug_data = {
                'points': new_points,
                'atomic_numbers': new_atomic_numbers,
                'x_vec': x_vec,
                'y_vec': y_vec,
                'z_vec': z_vec
            }
            with open('crash_input.pkl', 'wb') as f:
                pickle.dump(debug_data, f)
            # --- End of debugging save ---
            
            next_point = VoronoiGenerator._get_next_point(
                points=new_points,
                atomic_numbers=new_atomic_numbers,
                x_vec=x_vec,
                y_vec=y_vec,
                z_vec=z_vec
            )
            
            if np.any(np.isnan(next_point)):
                print(f"NaN value detected for a sample, skipping ghost atom generation for it.")
                break

            new_atomic_numbers = np.append(new_atomic_numbers, -1)
            new_points = np.vstack([new_points, next_point])

    # --- 3. Update the data object with the new atom info ---
    single_data.pos = torch.from_numpy(new_points).to(dtype=torch.float64)
    single_data.n_atoms = torch.tensor(len(new_points), dtype=torch.long)
    single_data.batch = torch.zeros(len(new_points), dtype=torch.long)
    
    # Update species efficiently
    single_data.species = torch.from_numpy(new_atomic_numbers).long()

    ghosted_val_data_list.append(single_data)

print(f"\n✅ Ghost atom generation complete. Total samples processed: {len(ghosted_val_data_list)}")


Starting ghost atom generation for 9047 samples...


  0%|          | 45/9047 [00:07<26:35,  5.64it/s]Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
  2%|▏         | 147/9047 [00:24<23:46,  6.24it/s]Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
  2%|▏         | 151/9047 [00:25<24:51,  5.96it/s]Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
  2%|▏         | 188/9047 [00:31<23:50,  6.19it/s]Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
  3%|▎         | 239/9047 [00:39<25:09,  5.84it/s]Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
Order 4 vertex memory scaled up to 16
  3%|▎         | 298/9047 [00:48<22:37,  6.44it/s]Order 4 vertex memory scaled up to 16
Order 4 vertex me


✅ Ghost atom generation complete. Total samples processed: 9047


In [21]:
# --- Define File Paths for the full validation set ---
from pathlib import Path
processed_dir = Path('./processed_datasets/')
processed_dir.mkdir(exist_ok=True) # Ensure the directory exists

full_val_xyz_filepath = processed_dir / 'val_ghosted.xyz'
full_val_lmdb_filepath = processed_dir / 'val_ghosted.lmdb'

# --- Execute the full saving pipeline for the new dataset ---
save_atoms_to_lmdb(ghosted_val_data_list, full_val_xyz_filepath, full_val_lmdb_filepath)



--- Starting save process: Atoms -> XYZ -> LMDB ---
Successfully saved intermediate file: 'processed_datasets/val_ghosted.xyz'
Read 9047 structures from 'processed_datasets/val_ghosted.xyz'
Successfully created single-file LMDB: 'processed_datasets/val_ghosted.lmdb'
--- Save process complete ---


In [10]:
from tqdm import tqdm
import numpy as np
import torch
import pickle
from omg.datamodule.dataloader import OMGData

# Debug..
with open('crash_input.pkl', 'rb') as f:
    debug_data = pickle.load(f)

species = torch.from_numpy(np.array(debug_data['atomic_numbers'])).long()
pos = torch.from_numpy(np.array(debug_data['points'])).float()
cell = torch.from_numpy(np.array([
    debug_data['x_vec'],
    debug_data['y_vec'],
    debug_data['z_vec']
])).float()

data_for_plotting = OMGData.from_data(
    species=species,
    pos=pos,
    cell=cell,
    convert_to_fractional=False 
)

print(data_for_plotting)

plot_crystal_with_points(data_for_plotting, 0)

OMGData(
  n_atoms=2,
  batch=[2],
  species=[2],
  cell=[1, 3, 3],
  pos=[2, 3],
  property={}
)


In [11]:
data_for_plotting

OMGData(
  n_atoms=2,
  batch=[2],
  species=[2],
  cell=[1, 3, 3],
  pos=[2, 3],
  property={}
)

In [12]:
# Remove the ghost atom (species == -1) from data_for_plotting
mask = data_for_plotting.species != -1
filtered_data = OMGData.from_data(
    species=data_for_plotting.species[mask],
    pos=data_for_plotting.pos[mask],
    cell=data_for_plotting.cell,
    convert_to_fractional=False
)

# Save to temp.pkl
import pickle
with open('temp.pkl', 'wb') as f:
    # filtered_data.cell has shape [1, 1, 3, 3], so we need to index it correctly
    # First [0] gets us [1, 3, 3], second [0] gets us [3, 3]
    cell_matrix = filtered_data.cell[0, 0]  # Get the [3, 3] matrix
    
    pickle.dump({
        'atomic_numbers': filtered_data.species.numpy(),
        'points': filtered_data.pos.numpy(),
        'x_vec': cell_matrix[0].numpy(),
        'y_vec': cell_matrix[1].numpy(),
        'z_vec': cell_matrix[2].numpy()
    }, f)

print(f"✅ Saved filtered data to temp.pkl (removed ghost atom with species=-1)")
print(filtered_data)
print(f"Cell shape: {filtered_data.cell.shape}")

✅ Saved filtered data to temp.pkl (removed ghost atom with species=-1)
OMGData(
  n_atoms=1,
  batch=[1],
  species=[1],
  cell=[1, 1, 3, 3],
  pos=[1, 3],
  property={}
)
Cell shape: torch.Size([1, 1, 3, 3])


In [13]:
from tqdm import tqdm
import numpy as np
import torch
import pickle
from omg.datamodule.dataloader import OMGData

# Debug..
with open('temp.pkl', 'rb') as f:
    debug_data = pickle.load(f)

species = torch.from_numpy(np.array(debug_data['atomic_numbers'])).long()
pos = torch.from_numpy(np.array(debug_data['points'])).float()
cell = torch.from_numpy(np.array([
    debug_data['x_vec'],
    debug_data['y_vec'],
    debug_data['z_vec']
])).float()

data_for_plotting = OMGData.from_data(
    species=species,
    pos=pos,
    cell=cell,
    convert_to_fractional=False 
)

print(data_for_plotting)

plot_crystal_with_points(data_for_plotting, 0)

OMGData(
  n_atoms=1,
  batch=[1],
  species=[1],
  cell=[1, 3, 3],
  pos=[1, 3],
  property={}
)


In [14]:
import pickle
import numpy as np
from voronoi_weighted import VoronoiPhantomCellGenerator

# Load the crash input data
with open('temp.pkl', 'rb') as f:
    crash_data = pickle.load(f)

points = crash_data['points']
atomic_numbers = crash_data['atomic_numbers']
x_vec = crash_data['x_vec']
y_vec = crash_data['y_vec']
z_vec = crash_data['z_vec']

print("✅ Successfully loaded 'crash_input.pkl'.")
print(f"Number of atoms in the cell: {len(points)}")

# Instantiate the generator
test_generator = VoronoiPhantomCellGenerator(desired_atom_count=len(points) + 1)

# Run the test
try:
    next_point = test_generator._get_next_point(
        points=points,
        atomic_numbers=atomic_numbers,
        x_vec=x_vec,
        y_vec=y_vec,
        z_vec=z_vec
    )
    print("✅ Test passed: _get_next_point executed successfully.")
    print(f"Generated point: {next_point}")
except Exception as e:
    print(f"An error occurred during the test: {e}")

# Save the extended data (with the new point) to temp_extended.pkl
extended_data = {
    'points': np.vstack([points, next_point]),
    'atomic_numbers': np.append(atomic_numbers, -1),  # Duplicate first atom's atomic number
    'x_vec': x_vec,
    'y_vec': y_vec,
    'z_vec': z_vec
}

with open('temp_extended.pkl', 'wb') as f:
    pickle.dump(extended_data, f)

print("Successfully saved extended data to 'temp_extended.pkl'.")



✅ Successfully loaded 'crash_input.pkl'.
Number of atoms in the cell: 1
✅ Test passed: _get_next_point executed successfully.
Generated point: [-1.1920929e-07  7.1525574e-07 -3.5762787e-07]
Successfully saved extended data to 'temp_extended.pkl'.


In [15]:
from tqdm import tqdm
import numpy as np
import torch
import pickle
from omg.datamodule.dataloader import OMGData

# Debug..
with open('temp_extended.pkl', 'rb') as f:
    debug_data = pickle.load(f)

species = torch.from_numpy(np.array(debug_data['atomic_numbers'])).long()
pos = torch.from_numpy(np.array(debug_data['points'])).float()
cell = torch.from_numpy(np.array([
    debug_data['x_vec'],
    debug_data['y_vec'],
    debug_data['z_vec']
])).float()

data_for_plotting = OMGData.from_data(
    species=species,
    pos=pos,
    cell=cell,
    convert_to_fractional=False 
)

print(data_for_plotting)

plot_crystal_with_points(data_for_plotting, 0)

OMGData(
  n_atoms=2,
  batch=[2],
  species=[2],
  cell=[1, 3, 3],
  pos=[2, 3],
  property={}
)


In [7]:
import pickle
import numpy as np
import plotly.graph_objects as go
from voronoi_weighted import VoronoiPhantomCellGenerator

# --- 1. Load the crash data ---
with open('temp.pkl', 'rb') as f:
    crash_data = pickle.load(f)

points = crash_data['points']
atomic_numbers = crash_data['atomic_numbers']
x_vec = np.array(crash_data['x_vec'])
y_vec = np.array(crash_data['y_vec'])
z_vec = np.array(crash_data['z_vec'])

print("✅ Data loaded from 'crash_input.pkl'.")

# --- 2. Instantiate the generator and create the supercell ---
# The desired_atom_count is not important here as we are just using the helper methods.
generator = VoronoiPhantomCellGenerator(desired_atom_count=1)

# Center the points before creating the supercell, mimicking the internal logic
center_vector = (x_vec + y_vec + z_vec) / 2
centered_points = points - center_vector
supercell_points = generator._create_supercell_from_points(centered_points, atomic_numbers, x_vec, y_vec, z_vec)

✅ Data loaded from 'crash_input.pkl'.


In [8]:
points

array([[1.7233326, 1.5865543, 1.8859483]], dtype=float32)

In [9]:
x_vec, y_vec, z_vec

(array([3.1839645, 0.       , 0.2874895], dtype=float32),
 array([0.26270077, 3.1731086 , 0.2874895 ], dtype=float32),
 array([0.       , 0.       , 3.1969178], dtype=float32))

In [10]:
center_vector

array([1.7233326, 1.5865543, 1.8859484], dtype=float32)

In [11]:
supercell_points

array([[-3.4466653e+00, -3.1731086e+00, -3.7718968e+00],
       [-3.4466653e+00, -3.1731086e+00, -5.7497913e-01],
       [-3.4466653e+00, -3.1731086e+00,  2.6219387e+00],
       [-3.1839645e+00,  0.0000000e+00, -3.4844074e+00],
       [-3.1839645e+00,  0.0000000e+00, -2.8748962e-01],
       [-3.1839645e+00,  0.0000000e+00,  2.9094281e+00],
       [-2.9212637e+00,  3.1731086e+00, -3.1969180e+00],
       [-2.9212637e+00,  3.1731086e+00, -1.1920929e-07],
       [-2.9212637e+00,  3.1731086e+00,  3.1969175e+00],
       [-2.6270077e-01, -3.1731086e+00, -3.4844074e+00],
       [-2.6270077e-01, -3.1731086e+00, -2.8748962e-01],
       [-2.6270077e-01, -3.1731086e+00,  2.9094281e+00],
       [ 0.0000000e+00,  0.0000000e+00, -3.1969180e+00],
       [ 0.0000000e+00,  0.0000000e+00, -1.1920929e-07],
       [ 0.0000000e+00,  0.0000000e+00,  3.1969175e+00],
       [ 2.6270077e-01,  3.1731086e+00, -2.9094286e+00],
       [ 2.6270077e-01,  3.1731086e+00,  2.8748938e-01],
       [ 2.6270077e-01,  3.1731

In [12]:
supercell_points.shape

(27, 3)

In [13]:
# --- 3. Compute all Voronoi vertices for the supercell ---
import freud
box_matrix = np.column_stack([3 * x_vec, 3 * y_vec, 3 * z_vec])
box = freud.box.Box.from_matrix(box_matrix)
voro = freud.locality.Voronoi()
cells = voro.compute((box, supercell_points)).polytopes
all_vertices = box.wrap(np.concatenate(cells))
voronoi_vertices = np.unique(np.round(all_vertices, decimals=10), axis=0)


print(f"Generated {len(supercell_points)} supercell points.")
print(f"Found {len(voronoi_vertices)} unique Voronoi vertices.")

Generated 27 supercell points.
Found 219 unique Voronoi vertices.


In [16]:

# --- 4. Visualization ---
fig = go.Figure()

# Add supercell points
fig.add_trace(go.Scatter3d(
    x=supercell_points[:, 0],
    y=supercell_points[:, 1],
    z=supercell_points[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.6),
    name='Supercell Atoms'
))

# Add Voronoi vertices
hover_texts = [f'({v[0]:.3f}, {v[1]:.3f}, {v[2]:.3f})' for v in voronoi_vertices]

# Add Voronoi vertices
fig.add_trace(go.Scatter3d(
    x=voronoi_vertices[:, 0],
    y=voronoi_vertices[:, 1],
    z=voronoi_vertices[:, 2],
    mode='markers',
    marker=dict(size=3, color='red', opacity=0.8),
    name='Voronoi Vertices',
    hovertext=hover_texts,
    hoverinfo='text'
))

# --- Draw the supercell box ---
super_x = 3 * x_vec
super_y = 3 * y_vec
super_z = 3 * z_vec

# The origin of the supercell box will be shifted because of the centering
box_origin = -1.5 * (x_vec + y_vec + z_vec)

vertices = [
    box_origin,
    box_origin + super_x,
    box_origin + super_y,
    box_origin + super_z,
    box_origin + super_x + super_y,
    box_origin + super_x + super_z,
    box_origin + super_y + super_z,
    box_origin + super_x + super_y + super_z
]

edges = [
    (0, 1), (0, 2), (0, 3), (1, 4), (1, 5), (2, 4), (2, 6),
    (3, 5), (3, 6), (4, 7), (5, 7), (6, 7)
]

for i in [-1, 0, 1]:
    for j in [-1, 0, 1]:
        for k in [-1, 0, 1]:
            # Calculate the origin of the subcell
            subcell_origin = (i - 0.5) * x_vec + (j - 0.5) * y_vec + (k - 0.5) * z_vec
            
            # Define the 8 vertices of the subcell parallelepiped
            subcell_vertices = [
                subcell_origin,
                subcell_origin + x_vec,
                subcell_origin + y_vec,
                subcell_origin + z_vec,
                subcell_origin + x_vec + y_vec,
                subcell_origin + x_vec + z_vec,
                subcell_origin + y_vec + z_vec,
                subcell_origin + x_vec + y_vec + z_vec
            ]

            # Determine line properties (thicker and solid for the central cell)
            if i == 0 and j == 0 and k == 0:
                line_props = dict(color='black', width=3)
            else:
                line_props = dict(color='grey', width=1, dash='dot')

            # Draw each edge
            for v_idx1, v_idx2 in edges:
                v1, v2 = subcell_vertices[v_idx1], subcell_vertices[v_idx2]
                fig.add_trace(go.Scatter3d(
                    x=[v1[0], v2[0]],
                    y=[v1[1], v2[1]],
                    z=[v1[2], v2[2]],
                    mode='lines',
                    line=line_props,
                    showlegend=False,
                    hoverinfo='none'
                ))


fig.update_layout(
    title='Supercell and Voronoi Vertices Visualization',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    width=800,
    height=800
)

fig.show()

In [20]:
# --- 1. Filter Voronoi vertices to the central cell ---
# This logic is adapted from the _get_voronoi_vertices method.

# The box_matrix for the 3x3x3 supercell
box_matrix = np.column_stack([3 * x_vec, 3 * y_vec, 3 * z_vec])

# Convert Cartesian coordinates of vertices to fractional coordinates of the UNIT cell
fractional_coords = np.linalg.solve(box_matrix / 3, voronoi_vertices.T).T

# Create a mask to find vertices within the central cell [-0.5, 0.5)
epsilon = 1e-3  # Using the same epsilon from your modified file
mask = np.all((fractional_coords >= -0.5 - epsilon) & (fractional_coords < 0.5 + epsilon), axis=1)
center_cell_vertices = voronoi_vertices[mask]

print(f"Found {len(center_cell_vertices)} vertices inside the central cell.")

# --- 2. Visualization of the Central Cell ---
fig_center = go.Figure()

# Add the original atoms in the central cell (using the 'centered_points' from before)
fig_center.add_trace(go.Scatter3d(
    x=centered_points[:, 0],
    y=centered_points[:, 1],
    z=centered_points[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.8),
    name='Central Cell Atoms'
))

# Prepare hover text for the filtered Voronoi vertices
hover_texts_center = [f'({v[0]:.3f}, {v[1]:.3f}, {v[2]:.3f})' for v in center_cell_vertices]

# Add the filtered Voronoi vertices
fig_center.add_trace(go.Scatter3d(
    x=center_cell_vertices[:, 0],
    y=center_cell_vertices[:, 1],
    z=center_cell_vertices[:, 2],
    mode='markers',
    marker=dict(size=3, color='red'),
    name='Central Voronoi Vertices',
    hovertext=hover_texts_center,
    hoverinfo='text'
))

# --- Draw the central cell boundary ---
cell_origin = -0.5 * (x_vec + y_vec + z_vec)
cell_vertices = [
    cell_origin, cell_origin + x_vec, cell_origin + y_vec, cell_origin + z_vec,
    cell_origin + x_vec + y_vec, cell_origin + x_vec + z_vec,
    cell_origin + y_vec + z_vec, cell_origin + x_vec + y_vec + z_vec
]
edges = [
    (0, 1), (0, 2), (0, 3), (1, 4), (1, 5), (2, 4), (2, 6),
    (3, 5), (3, 6), (4, 7), (5, 7), (6, 7)
]

for i, j in edges:
    v1, v2 = cell_vertices[i], cell_vertices[j]
    fig_center.add_trace(go.Scatter3d(
        x=[v1[0], v2[0]], y=[v1[1], v2[1]], z=[v1[2], v2[2]],
        mode='lines', line=dict(color='black', width=3),
        showlegend=False, hoverinfo='none'
    ))

fig_center.update_layout(
    title='Central Cell with its Voronoi Vertices',
    scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'),
    width=700, height=700
)

fig_center.show()

Found 12 vertices inside the central cell.
